# Crawling

- 웹이라는 애를 탐험하는 친구 지금까지는 단일페이지를 가져오는 정도였다.

- Web indexing이 목적인데 정확하게는 어떤단어가 어떤웹사이트나 문단에서 사용되는지 찾는거

- 시작 URL로 부터 반복적인 프로세스로 hyperlink를 찾고 fetching 한다 

- ppt에 crawling 구조 보기(굉장히 복잡) -> 이거를 코드로 만든다

- 다른데서 Crawler를 만들어봤다 할려면 이정도는 해야해

In [2]:
import requests
from bs4 import BeautifulSoup
import time

download 함수정의, headers 에 user-agent를 넣어줘

In [3]:
def download(method ,url, param=None, data=None,timeout=1, maxetries=3): ##없을수도 있는 것에 None // header 는 빠진형태
    try : 
        resp = requests.request(method, url, params=param, data=data, headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500<= e.response.status_code <600 and maxetries>0:
            print(maxetries)
            time.sleep(timeout)
            print("재시도")
            download(method,url,param,data,timeout,maxetries-1)
        else :
            print(e.response.status_code)
            print(e.response.reason)
    return resp

#### webscraping 사이트 예제

In [4]:
url = "http://example.webscraping.com/"

html = download("get", url)

In [5]:
dom = BeautifulSoup(html.text, "lxml")

3글자 미만인것들은 뭔가 문제가있는것들 처리 url join으로 완벽한 url 주소 만들어줬어.

In [7]:
[requests.compat.urljoin(url, _["href"]) for _ in dom.find_all("a") if len(_["href"]) > 3]

['http://example.webscraping.com/places/default/user/register?_next=/places/default/index',
 'http://example.webscraping.com/places/default/user/login?_next=/places/default/index',
 'http://example.webscraping.com/places/default/index',
 'http://example.webscraping.com/places/default/search',
 'http://example.webscraping.com/places/default/view/Afghanistan-1',
 'http://example.webscraping.com/places/default/view/Aland-Islands-2',
 'http://example.webscraping.com/places/default/view/Albania-3',
 'http://example.webscraping.com/places/default/view/Algeria-4',
 'http://example.webscraping.com/places/default/view/American-Samoa-5',
 'http://example.webscraping.com/places/default/view/Andorra-6',
 'http://example.webscraping.com/places/default/view/Angola-7',
 'http://example.webscraping.com/places/default/view/Anguilla-8',
 'http://example.webscraping.com/places/default/view/Antarctica-9',
 'http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10',
 'http://example.webscr

#### parseURL
parseURL 이라는 함수를 정의해주자 이 함수는 download를 통해 get방식으로 seed(url) 의 html 을 끌어온후 bs4 로 domtree 만들어주고, 마지막에는 url과 href를 참조하는 주소를 join하여 해당 웹사이트의 주소를 return 하는 함수.

In [8]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(url, _["href"]) for _ in dom.find_all("a") if len(_["href"]) > 3]
    

In [12]:
import time
queue = []
queue.append("http://example.webscraping.com/")
seen = []

while queue:
    baseURL = queue.pop(0) ##선입 선출.
    seen.append(baseURL)
    time.sleep(1) #슬립
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link) ##중복을 방지하기 위해 
    print("queue: {0}, Seen: {1}".format(len(queue), len(seen))) ##format 설명해줄때는 중괄호.!
    


queue: 15, Seen: 1
queue: 16, Seen: 2
queue: 15, Seen: 3
queue: 14, Seen: 4
queue: 15, Seen: 5
queue: 24, Seen: 6
queue: 28, Seen: 7
queue: 36, Seen: 8
queue: 46, Seen: 9
queue: 49, Seen: 10
queue: 53, Seen: 11
queue: 59, Seen: 12
queue: 62, Seen: 13
queue: 65, Seen: 14
queue: 67, Seen: 15
queue: 80, Seen: 16
queue: 79, Seen: 17
queue: 78, Seen: 18


KeyboardInterrupt: 

- queue가 의미하는것?
큐는 처음에서 pop되면서 줄어듬
- seen이 의미하는것?
씬은 결국 본것들 !!

#### 구글, 박보영 예제

In [23]:
url = "https://www.google.com/search"
param = { "q" : "박보영"}
headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
html = download("get",  url, param)
dom = BeautifulSoup(html.text, "lxml")

박보영 검색결과 타이틀만 가져왔다. 돔트리 구조가 간단하기때문에 쉽게 가져옴

In [24]:
dom.select(".LC20lb")

[<h3 class="LC20lb">박보영 - 나무위키</h3>,
 <h3 class="LC20lb">박보영 - 위키백과, 우리 모두의 백과사전</h3>,
 <h3 class="LC20lb">박보영의 작품 목록 - 위키백과, 우리 모두의 백과사전</h3>,
 <h3 class="LC20lb">박보영은 오래 지켜본다. 연애도, 연기 변신도 - 중앙일보</h3>,
 <h3 class="LC20lb">박보영 - Daum 영화</h3>,
 <h3 class="LC20lb">전체글 - 박보영 공식 팬카페 _ 뽀르테 - Daum 카페</h3>,
 <h3 class="LC20lb">[HD스토리] 작은 거인 박보영…그의 연기에 대한 흔적들 - MSN.com</h3>,
 <h3 class="LC20lb">#박보영 hashtag on Twitter</h3>,
 <h3 class="LC20lb">박보영 - 신나라레코드</h3>]

In [25]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(url, _["href"]) for _ in dom.find_all("a") if _.has_attr("href") and len(_["href"]) > 3]
## href를 attribute 하고, len이 3이상인것.

In [26]:
queue = []
queue.extend([_.find_parent()["href"] for _ in dom.select(".LC20lb")])
seen = []

while queue:
    baseURL = queue.pop(0) ##선입 선출.
    seen.append(baseURL)
    #time.sleep(1) #슬립
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link) ##중복을 방지하기 위해 
    print("queue: {0}, Seen: {1}".format(len(queue), len(seen)))

queue: 983, Seen: 1
queue: 1836, Seen: 2
queue: 2093, Seen: 3
queue: 2215, Seen: 4
queue: 2258, Seen: 5
queue: 2312, Seen: 6
queue: 2351, Seen: 7
queue: 2535, Seen: 8
queue: 2691, Seen: 9
404
Not Found
queue: 2691, Seen: 10
404
Not Found
queue: 2690, Seen: 11
404
Not Found
queue: 2689, Seen: 12
queue: 2756, Seen: 13
404
Not Found
queue: 2755, Seen: 14
404
Not Found
queue: 2754, Seen: 15
404
Not Found
queue: 2753, Seen: 16
404
Not Found
queue: 2752, Seen: 17
404
Not Found
queue: 2751, Seen: 18
404
Not Found
queue: 2750, Seen: 19
404
Not Found
queue: 2749, Seen: 20
404
Not Found
queue: 2748, Seen: 21
404
Not Found
queue: 2747, Seen: 22
404
Not Found
queue: 2746, Seen: 23
queue: 2810, Seen: 24
queue: 3135, Seen: 25
queue: 3169, Seen: 26
queue: 3203, Seen: 27
queue: 3246, Seen: 28
queue: 3288, Seen: 29
queue: 3332, Seen: 30
queue: 3376, Seen: 31
queue: 3421, Seen: 32
queue: 3464, Seen: 33
queue: 3506, Seen: 34
queue: 3555, Seen: 35
queue: 3587, Seen: 36
404
Not Found
queue: 3586, Seen: 37


KeyboardInterrupt: 

In [27]:
seen

['https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%EC%9D%98_%EC%9E%91%ED%92%88_%EB%AA%A9%EB%A1%9D',
 'https://news.joins.com/article/22895953',
 'https://movie.daum.net/person/main?personId=105244',
 'http://m.cafe.daum.net/parkboyoungfd',
 'https://www.msn.com/ko-kr/entertainment/tv/hd%EC%8A%A4%ED%86%A0%EB%A6%AC-%EC%9E%91%EC%9D%80-%EA%B1%B0%EC%9D%B8-%EB%B0%95%EB%B3%B4%EC%98%81%E2%80%A6%EA%B7%B8%EC%9D%98-%EC%97%B0%EA%B8%B0%EC%97%90-%EB%8C%80%ED%95%9C-%ED%9D%94%EC%A0%81%EB%93%A4/ar-BBUvL0V',
 'https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81',
 'http://www.synnara.co.kr/sp/SP900.do?topmenu=N&mem_id=D000219965',
 'https://www.google.com/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C%80%EB%AC%B8',
 'https://www.google.com/RecentChanges',
 'https://www.google.com/RecentDiscuss',
 'https://board.namu.wiki/',
 'https://www.google.com/NeededPages',
 'https://www.goo

### focused crawling
####  naver

In [28]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(url, _["href"]) for _ in dom.find_all("a") if len(_["href"]) > 3]

In [29]:
url = "https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EB%B0%95%EB%B3%B4%EC%98%81"
html = download("get", url)
dom = BeautifulSoup(html.text, "lxml")

In [31]:
[_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title") ]

['https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950',
 'https://asovc7777.blog.me/221548676988',
 'http://hshlog.com/221559890110',
 'https://blog.naver.com/js2y86?Redirect=Log&logNo=221530397718',
 'https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195']

In [33]:
### 체크하는 함수 정의
## 
def checkBlog(url):
    return requests.compat.urlparse(url)[1] == "blog.naver.com"

In [32]:
requests.compat.urlparse(url)
##얘를 보면 1번쨰 인덱스가 search.naver.com 혹은 블로그이면 blog.naver.com 이런식이다!

ParseResult(scheme='https', netloc='search.naver.com', path='/search.naver', params='', query='sm=top_hty&fbm=1&ie=utf8&query=%EB%B0%95%EB%B3%B4%EC%98%81', fragment='')

#### 네이버의 경우에 돔안에 돔이 싸여있는 구조이다 즉 메인프레임에 갇혀있다.

In [ ]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    if len(dom.select("#mainFrame")) < 1: ##즉 메인프레임에 싸여있지 않으면.
        return [] ## 빈 리스트를 리턴
    ## 메인프레임에 싸여있다면, mainframe의 src를 이용하자.
    seed = requests.compat.urljoin(seed
                                   , dom.select("#mainFrame")[0]["src"])
    
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(seed, _["href"]) 
            for _ in dom.find_all("a") 
            if _.has_attr("href") and len(_["href"]) > 3 and checkBlog(requests.compat.urljoin(seed, _["href"]))]

In [35]:
[_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title") ]

queue = []
queue.append("https://search.naver.com/search.naver?where=post&sm=tab_jum&query=%EC%86%90%ED%9D%A5%EB%AF%BC")
seen = []

while queue:
    baseURL = queue.pop(0) ##선입 선출.
    seen.append(baseURL)
    #time.sleep(1) #슬립
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link) ##중복을 방지하기 위해 
    print("queue: {0}, Seen: {1}".format(len(queue), len(seen)))
    
## 자바 스크립트 오류 

queue: 167, Seen: 1
queue: 317, Seen: 2
queue: 316, Seen: 3
queue: 511, Seen: 4
queue: 522, Seen: 5
queue: 528, Seen: 6
queue: 527, Seen: 7
queue: 526, Seen: 8
queue: 526, Seen: 9


InvalidSchema: No connection adapters were found for 'javascript:;'

In [36]:
seen

['https://search.naver.com/search.naver?where=post&sm=tab_jum&query=%EC%86%90%ED%9D%A5%EB%AF%BC',
 'https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EB%B0%95%EB%B3%B4%EC%98%81#lnb',
 'https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EB%B0%95%EB%B3%B4%EC%98%81#content',
 'https://www.naver.com',
 'https://help.naver.com/support/alias/search/word/word_16.naver',
 'https://nid.naver.com/nidlogin.login?url=https%3A%2F%2Fsearch.naver.com%2Fsearch.naver%3Fwhere%3Dpost%26sm%3Dtab_jum%26query%3D%25EC%2586%2590%25ED%259D%25A5%25EB%25AF%25BC',
 'https://help.naver.com/support/alias/search/word/word_21.naver',
 'https://help.naver.com/support/alias/search/word/word_17.naver',
 'https://help.naver.com/support/alias/search/word/word_18.naver',
 'javascript:;']

### 다음 블로그 가져오기

In [38]:
#다음 블로그들을 가져오기 위한 코드
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    return [requests.compat.urljoin(seed, _["href"])
 for _ in dom.find_all("a") 
        if _.has_attr("href") and len(_["href"]) > 3
            and checkBlog(requests.compat.urljoin(seed, _["href"]))]

In [37]:
#tistory.com과 매칭? 
def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")

In [39]:
#daum 주소 설정
url = "https://search.daum.net/search"
#&w=tot&DA=SBC&q=손흥민
data = {
    "q":"손흥민",
       }
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
html = download("get",url,data,headers)
dom = BeautifulSoup(html.text, "lxml")

[_["href"] for _ in dom.select("a.f_link_b") if checkBlog(_["href"]) ]

['http://g-in0519.tistory.com/50',
 'http://pjs2002.tistory.com/150',
 'http://pigbill.tistory.com/12',
 'http://itlionaire.tistory.com/98']

In [40]:
while queue:
    baseURL = queue.pop(0) ##선입 선출.
    seen.append(baseURL)
    #time.sleep(1) #슬립
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link) ##중복을 방지하기 위해 
    print("queue: {0}, Seen: {1}".format(len(queue), len(seen)))

queue: 524, Seen: 11
queue: 523, Seen: 12
queue: 522, Seen: 13
queue: 521, Seen: 14
queue: 520, Seen: 15
queue: 519, Seen: 16
queue: 518, Seen: 17
queue: 517, Seen: 18
queue: 516, Seen: 19
queue: 515, Seen: 20
queue: 514, Seen: 21
queue: 513, Seen: 22
queue: 512, Seen: 23
queue: 511, Seen: 24
queue: 510, Seen: 25
queue: 509, Seen: 26
queue: 508, Seen: 27
queue: 507, Seen: 28
queue: 506, Seen: 29
queue: 505, Seen: 30
queue: 504, Seen: 31
queue: 503, Seen: 32
queue: 502, Seen: 33
queue: 501, Seen: 34
queue: 500, Seen: 35
queue: 499, Seen: 36
queue: 498, Seen: 37
queue: 497, Seen: 38
queue: 496, Seen: 39
queue: 495, Seen: 40
queue: 494, Seen: 41
queue: 493, Seen: 42
queue: 492, Seen: 43
queue: 491, Seen: 44
queue: 490, Seen: 45
queue: 489, Seen: 46
queue: 488, Seen: 47
queue: 487, Seen: 48
queue: 486, Seen: 49
queue: 485, Seen: 50
queue: 484, Seen: 51
queue: 483, Seen: 52
queue: 482, Seen: 53
queue: 481, Seen: 54
queue: 480, Seen: 55
queue: 479, Seen: 56
queue: 478, Seen: 57
queue: 477, S

KeyboardInterrupt: 

In [41]:
seen

['https://search.naver.com/search.naver?where=post&sm=tab_jum&query=%EC%86%90%ED%9D%A5%EB%AF%BC',
 'https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EB%B0%95%EB%B3%B4%EC%98%81#lnb',
 'https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EB%B0%95%EB%B3%B4%EC%98%81#content',
 'https://www.naver.com',
 'https://help.naver.com/support/alias/search/word/word_16.naver',
 'https://nid.naver.com/nidlogin.login?url=https%3A%2F%2Fsearch.naver.com%2Fsearch.naver%3Fwhere%3Dpost%26sm%3Dtab_jum%26query%3D%25EC%2586%2590%25ED%259D%25A5%25EB%25AF%25BC',
 'https://help.naver.com/support/alias/search/word/word_21.naver',
 'https://help.naver.com/support/alias/search/word/word_17.naver',
 'https://help.naver.com/support/alias/search/word/word_18.naver',
 'javascript:;',
 'https://search.naver.com/search.naver?where=nexearch&sm=tab_jum&query=%EC%86%90%ED%9D%A5%EB%AF%BC',
 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query=%EC%86%90%ED%9D%A5%EB%AF%BC',

depth가 있는 parseURL

## DB 만들기

#### 문자열을(여기선 URL)을 PK값으로 삼으면 난리남 절대 하지마

id url seen data

DB 구조는

- table1 = id, path, param, seen, date

- table2 = id, netloc, date

- url = netloc + path + param

#### 테이블 두개 생성

In [42]:
import sqlite3
con = sqlite3.connect("idontknow3.db")
cur = con.cursor()

In [48]:
cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table2_id INTEGER NOT NULL,
        path TEXT NOT NULL,
        param TEXT NOT NULL,
        seen BOOLEAN DEFAULT FALSE NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
        );""")
#새로정의 할때마다 DROP 해주기위해서

In [49]:
cur.executescript("""
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc TEXT NOT NULL,
        data TIMESTAMP DEFAULT CURRENT_TIMSESTAMP NOT NULL
        );
        """)
con.commit()

In [53]:
## _urlparse라는건 이런거
requests.compat.urlparse(href)

ParseResult(scheme='http', netloc='m.cafe.daum.net', path='/parkboyoungfd', params='', query='', fragment='')

In [50]:
# seed URL -> DB Insert
url = "https://www.google.com/search"
param = {"q":"박보영"}
html = download("get",url,param)
dom = BeautifulSoup(html.text, "lxml")

for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
    _urlparse = requests.compat.urlparse(href)
    netloc = "://".join(_urlparse[:2])
    cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
    
    netlocID = cur.fetchone()
    if not netlocID:
        cur.execute("INSERT INTO table2(netloc) VALUES(?)",[netloc])
        con.commit()
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()
        
        cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)",
                   [netlocID[0], _urlparse[2], _urlparse[4]])
        con.commit()

In [ ]:
while True:
    cur.execute("""
        SELECT table1.id, table2.netloc, table1.path, table1.param
        FROM table1
        JOIN table2 ON table1.table2_id=table2.id
        WHERE table1.seen='FALSE'
        ORDER BY table1.date ASC
        LIMIT 0, 1; 
        """)
    
    seed = cur.fetchone()
    if not seed:
        break;
    print(seed)
    print("{0}{1}?{2}".format(seed[1],seed[2],seed[3]))
    break
    

    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
        _urlparse = requests.compat.urlparse(href)
        netloc = "://".join(_urlparse[:2])
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
    
    netlocID = cur.fetchone()
    if not netlocID:
        cur.execute("INSERT INTO table2(netloc) VALUES(?)",[netloc])
        con.commit()
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()
        
        cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)",
                   [netlocID[0], _urlparse[2], _urlparse[4]])
        con.commit()